<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 4: Introduction to Data Fitting</h1>


<a name='section_4_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L4.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_4_1">L4.1 An Example: Hubble Constant</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_4_1">L4.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_4_2">L4.2 Derivation of Linear Regression</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_4_2">L4.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_4_3">L4.3 Linear Regression: Coding Example</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_4_3">L4.3 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_4_4">L4.4 Weighted Linear Regression</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_4_4">L4.4 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_4_5">L4.5 Minimization without the Math</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_4_5">L4.5 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_4_6">L4.6 Gradient Descent</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_4_6">L4.6 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_4_7">L4.7 Fitting the Full Range of Hubble Data</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_4_7">L4.7 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_4_8">L4.8 Fitting with lmfit</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_4_8">L4.8 Exercises</a></td>
    </tr>
</table>



In [ ]:
#>>>RUN: L4.0-runcell01

#importing data from git repository

!git init
!git remote add -f origin https://github.com/mitx-8s50/nb_LEARNER/
!git config core.sparseCheckout true
!echo 'data/L04' >> .git/info/sparse-checkout
!git pull origin main

In [ ]:
#>>>RUN: L4.0-runcell02

!pip install lmfit

In [ ]:
#>>>RUN: L4.0-runcell03

#import packages
import math as math                   #https://docs.python.org/3/library/math.html
import matplotlib.pyplot as plt       #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
import numpy as np                    #https://numpy.org/doc/stable/
import csv                            #https://docs.python.org/3/library/csv.html 
from scipy import stats               #https://docs.scipy.org/doc/scipy/reference/stats.html
from scipy.optimize import curve_fit  #https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html
import scipy.linalg as la             #https://docs.scipy.org/doc/scipy/reference/linalg.html
import lmfit                          #https://lmfit.github.io/lmfit-py/ 

In [ ]:
#>>>RUN: L4.0-runcell04

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title

<a name='section_4_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L4.1 An Example: Hubble Constant</h2>  

| [Top](#section_4_0) | [Previous Section](#section_4_0) | [Exercises](#exercises_4_1) | [Next Section](#section_4_2) |


In [ ]:
#>>>RUN: L4.1-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L04/slides_L04_01.html', width=975, height=550)

<a name='exercises_4_1'></a>     

| [Top](#section_4_0) | [Restart Section](#section_4_1) | [Next Section](#section_4_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-4.1.1 Expansion and Recessional Velocity</span>

True or False: If the universe is expanding, the apparent recessional velocity of galaxies further away will be larger.


<a name='section_4_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L4.2 Derivation of Linear Regression</h2>  

| [Top](#section_4_0) | [Previous Section](#section_4_1) | [Exercises](#exercises_4_2) | [Next Section](#section_4_3) |


In [ ]:
#>>>RUN: L4.2-runcell01

import math as math
import matplotlib.pyplot as plt
import numpy as np
import csv

#Today we are going to start with astro data from here : 
#http://supernova.lbl.gov/Union/
#Let's load the data

label='data/L04/sn_z_mu_dmu_plow_union2.1.txt'


#Table stores name, redshift, distance modulus, distance modulus error
#Let's convert from distance modulus to distance
#See here https://en.wikipedia.org/wiki/Distance_modulus
def distanceconv(iMu):
    power=iMu/5+1
    return 10**power

# and the same for uncertainty in distance modulus to uncertainty in distance
def distanceconverr(iMu,iMuErr):
    power=iMu/5+1
    const=math.log(10)/5.
    return const*(10**power)*iMuErr

redshift=[]
distance=[]
distance_err=[]
# read in the data
with open(label,'r') as csvfile:
    plots = csv.reader(csvfile, delimiter='\t')
    # name, redshift, distance modulus, distance modulus uncertainty
    for row in plots:
        redshift.append(float(row[1]))
        distance.append(distanceconv(float(row[2])))
        distance_err.append(distanceconverr(float(row[2]),float(row[3])))

plt.xlabel('redshift(z)', fontsize=15) #Label x
plt.ylabel('distances(parsec)', fontsize=15)#Label y
# plot with errorbars
plt.errorbar(redshift,distance,yerr=distance_err,marker='.',linestyle = 'None', color = 'black')
plt.show()

In [ ]:
#>>>RUN: L4.2-runcell02

#Now let's zoom in on the small redshift data.
def load(iLabel,iZMax):
    # loads data from file iLabel with redshift below iZMax:
    redshift=[]
    distance=[]
    distance_err=[]
    with open(label,'r') as csvfile:
        plots = csv.reader(csvfile, delimiter='\t')
        for row in plots:
            if float(row[1]) > iZMax:
                continue
            redshift.append(float(row[1]))
            distance.append(distanceconv(float(row[2])))
            distance_err.append(distanceconverr(float(row[2]),float(row[3])))
    return np.array(redshift), np.array(distance), np.array(distance_err)

redshift,distance,distance_err = load(label,0.1)
plt.xlabel('redshift(z)', fontsize=15) #Label x
plt.ylabel('distances(parsec)', fontsize=15)#Label y
plt.errorbar(redshift,distance,yerr=distance_err,marker='.',linestyle = 'None', color = 'black')
plt.show()

<a name='exercises_4_2'></a>     

| [Top](#section_4_0) | [Restart Section](#section_4_2) | [Next Section](#section_4_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-4.2.1 Complete the Derivation</span>

In the derivation shown previously, how did we get the strange formulation in that second step? Part of the answer is that we could add a term $\sum_{i} x_{i}\bar{x} - \bar{x}^2$ to the denominator because $\sum_{i} x_{i}\bar{x} - \bar{x}^2 = 0$. This helped simplify the derivation. Below, we show a partial proof that $\sum_{i} x_{i}\bar{x} - \bar{x}^2 = 0$:

<br>

$$
\begin{eqnarray} 
\sum_{i} (x_{i}\bar{x}-\bar{x}^{2}) & = & \sum_{i} x_{i}\bar{x} -\sum_{i} \bar{x}^{2} \\
& = & [\mathrm{insert\,missing\,step}]\\
& = & 0 \\
\end{eqnarray}
$$  

<br>

In order for this to be true, what must the value of $\sum_{i} x_{i}\bar{x}$ be? Express your answer in terms of `N` for $N$ and `xbar` for $\bar{x}$.


<a name='section_4_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L4.3 Linear Regression: Coding Example</h2>  

| [Top](#section_4_0) | [Previous Section](#section_4_2) | [Exercises](#exercises_4_3) | [Next Section](#section_4_4) |


In [ ]:
#>>>RUN: L4.3-runcell01

#Let's run the regression again
def variance(isamples):
    mean=isamples.mean()
    n=len(isamples)
    tot=0
    for pVal in isamples:
        tot+=(pVal-mean)**2
    return tot/n

def covariance(ixs,iys):
    meanx=ixs.mean()
    meany=iys.mean()
    n=len(ixs)
    tot=0
    for i0 in range(len(ixs)):
        tot+=(ixs[i0]-meanx)*(iys[i0]-meany)
    return tot/n

def linear(ix,ia,ib):
    return ia*ix+ib

def regress(redshift,distance):
    #Let's regress
    var=variance(redshift)
    cov=covariance(redshift,distance)
    A=cov/var
    b=distance.mean()-A*redshift.mean()
    #Done!
    return A,b

def plotAll(redshift,distance,distance_err,A,b):
    #now let's plot it
    xvals = np.linspace(0,0.1,100)
    yvals = []
    for pX in xvals:
        yvals.append(linear(pX,A,b))

    #Plot the line
    plt.plot(xvals,yvals)
    plt.errorbar(redshift,distance,yerr=distance_err,marker='.',linestyle = 'None', color = 'black')
    plt.xlabel('redshift(z)', fontsize=15) #Label x
    plt.ylabel('distances(parsec)', fontsize=15)#Label y
    plt.show()
    #Print it out
    print("Hubbles Constant:",1e6*3e5/A,"intercept",b)#Note 1e6 is from pc to Mpc and 3e5 is c in km/s

A,b=regress(redshift,distance)
plotAll(redshift,distance,distance_err,A,b)

<a name='exercises_4_3'></a>     

| [Top](#section_4_0) | [Restart Section](#section_4_3) | [Next Section](#section_4_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-4.3.1 Uncertainty in $h_{0}$</span>

Now we wish to obtain the uncertainty on $h_{0}$, which we call $\sigma_{h_0}$, where we will use the definition:

$$h_{0} =\frac{1}{A}\frac{\rm 3\times10^{5}~(km/s)}{10^{-6}} $$


Assume that we know the uncertainty on $A$, which we call $\sigma_{A}$. What is the value of $\sigma_{h_0}$? Express your answer in terms of `A` for $A$, `h_0` for $h_{0}$, and `sigma_A` for $\sigma_{A}$.

**Hint: To answer this question, you need to use propagation of uncertainty!**

>#### Follow-up 4.3.1a (ungraded)
>
>What is the uncertainty $\sigma_{h_0}$ written only in terms of the slope $A$ and its uncertainty? 


<a name='section_4_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L4.4 Weighted Linear Regression</h2>  

| [Top](#section_4_0) | [Previous Section](#section_4_3) | [Exercises](#exercises_4_4) | [Next Section](#section_4_5) |


In [ ]:
#>>>RUN: L4.4-runcell01

weights=[]
for err in distance_err:
    weights.append(1./err**2)
    
weights = np.array(weights)

#Now let's do it with weights
def variance_w(isamples,iweights):
    mean=np.average(isamples,weights=iweights)
    sumw=np.sum(iweights)
    tot=0
    for i0 in range(len(isamples)):
        tot+=iweights[i0]*(isamples[i0]-mean)**2
    return tot/sumw

def covariance_w(ixs,iys,iweights):
    meanx=np.average(ixs,weights=iweights)
    meany=np.average(iys,weights=iweights)
    sumw=np.sum(iweights)
    tot=0
    for i0 in range(len(ixs)):
        tot+=iweights[i0]*(ixs[i0]-meanx)*(iys[i0]-meany)
    return tot/sumw

def regress_w(redshift,weights,distance):
    varw=variance_w(redshift,weights)
    covw=covariance_w(redshift,distance,weights)
    Aw=covw/varw
    bw=np.average(distance,weights=weights)-Aw*np.average(redshift,weights=weights)
    return Aw,bw

Aw,bw=regress_w(redshift,weights,distance)
plotAll(redshift,distance,distance_err,Aw,bw)

<a name='exercises_4_4'></a>     

| [Top](#section_4_0) | [Restart Section](#section_4_4) | [Next Section](#section_4_5) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-4.4.1 Effects of Changing Uncertainty I</span>

Let's imagine, for no particular reason, that our uncertainty on any given observation is multiplied by a factor of the redshift. That is, we define a new $\sigma_{i}^{new} = z \sigma_{i}^{old}$. How would the Hubble constant found by fitting the data with these modified uncertainties change, compared to the case when $\sigma_{i}^{old}$ was used? Choose from the following options:

- The fitted value of the Hubble constant would increase
- The fitted value of the Hubble constant would decrease
- The fitted value of the Hubble constant would stay the same

You can modify the starting code below to find the answer.

In [ ]:
#>>>EXERCISE: L4.4.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#The code below scales the uncertainties being used in our weighted regression. 

import numpy as np

redshift,distance,distance_err = load(label,0.1)

# scale the errors by the redshift
new_errors = #YOUR CODE HERE

# translate these scaled uncertainties into new weights for regression (1/sigma^2 for each)
weights = 1 / (new_errors * new_errors)

Aw,bw=regress_w(redshift,weights,distance)
plotAll(redshift,distance,new_errors,Aw,bw)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-4.4.2 Effects of Changing Uncertainty II</span>

What happens if I made a mistake and all of the errors are 100 times larger than what I thought, such that $\sigma_{i}^{new} = 100 \sigma_{i}^{old}$? How would the Hubble constant found by fitting the data with these modified uncertainties change, compared to the case when $\sigma_{i}^{old}$ was used?

Again, choose from the following options:

- The fitted value of the Hubble constant would increase
- The fitted value of the Hubble constant would decrease
- The fitted value of the Hubble constant would stay the same

You can modify the starting code below to find the answer.

In [ ]:
#>>>EXERCISE: L4.4.2
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

import numpy as np

redshift,distance,distance_err = load(label,0.1)

# scale the errors by the redshift
new_errors = #YOUR CODE HERE

# translate these scaled uncertainties into new weights for regression (1/sigma^2 for each)
weights = 1 / (new_errors * new_errors)

Aw,bw=regress_w(redshift,weights,distance)
plotAll(redshift,distance,new_errors,Aw,bw)

<a name='section_4_5'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L4.5 Minimization without the Math</h2>  

| [Top](#section_4_0) | [Previous Section](#section_4_4) | [Exercises](#exercises_4_5) | [Next Section](#section_4_6) |


In [ ]:
#>>>RUN: L4.5-runcell01

from scipy import stats

#Now lets do the same thing with scipy
slope, intercept, r_value, p_value, std_err = stats.linregress(redshift,distance)
print("UnWeighted Fit:","Hubbles Constant:",1e6*3e5/slope,"intercept",intercept)

#Now the weighted version; we use scikit-learn, a package dedicated to fitting
from sklearn.linear_model import LinearRegression
model = LinearRegression()
redshifthack=np.reshape(redshift,(len(redshift),1))#line to get the fit code to work
model.fit(redshifthack,distance,weights)
slope=model.coef_
const=model.intercept_
print("Weighted Fit:","Hubbles Constant:",1e6*3e5/slope,"intercept",const)

In [ ]:
#>>>RUN: L4.5-runcell02

#scipy provides the general tool that optimizes
from scipy import optimize as opt

#First an example minimizing a simple polynomial
def f(x):
    return x**4 + 3*(x-2)**3 - 15*(x)**2 + 1
sol=opt.minimize_scalar(f, method='Brent')
x = np.linspace(-8, 5, 100)

plt.xlabel('x', fontsize=15) #Label x
plt.ylabel('f(x)', fontsize=15)#Label y
plt.plot(x, f(x));
plt.axvline(sol.x, c='red')
plt.show()

In [ ]:
#>>>RUN: L4.5-runcell03

# scipy offers a tool for fitting a generic curve to data.
# It's using the same sort of optimizer, to optimize its choice of error metric
from scipy.optimize import curve_fit 

# We define a curve functional form
def f(x,a,b):
    return a*x+b

popt, pcov = curve_fit(f, redshift,distance) # returns optimal values of a, b in popt
perr = np.sqrt(np.diag(pcov))
print("Unweighted Hubbles Constant:",1e6*3e5/popt[0],"+/-",(1e6*3e5/popt[0]/popt[0])*perr[0],"intercept",popt[1],"+/-",perr[1])

#Now let's do it weighted by uncertainties, by passing them in the "sigma" optional argument
popt, pcov = curve_fit(f, redshift,distance,sigma=distance_err)
perr = np.sqrt(np.diag(pcov))
print("Weighted Hubbles Constant:",1e6*3e5/popt[0],"+/-",(1e6*3e5/popt[0]/popt[0])*perr[0],"intercept",popt[1],"+/-",perr[1])

<a name='exercises_4_5'></a>     

| [Top](#section_4_0) | [Restart Section](#section_4_5) | [Next Section](#section_4_6) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-4.5.1 Finding the Minimum of a Potential</span>

One of the most important distributions used in high energy physics is the Mexcian Hat Potential distribution. A generic form is given by: 

$$
\begin{eqnarray} 
  V(x,y) & = & (x^2+y^2)^2-A(x^2+y^2)
\end{eqnarray}
$$

Here, let's consider a 1D version where we fix $y$ and $A$, so we can write this as: 

$$
\begin{equation}
f(x) = (x-2)^{2}(x+2)^2-24x^2
\end{equation}
$$

**Complete the code below to find ONE minimum of this new potential, using bounded minimization. What is the value of the minimum that you obtained? Enter your answer as a number with precision 1e-3.**

*Note, in the next video we will look at a potential with slightly different values, and discuss some aspects of the minimization.*

*Check out the `scipy.optimize.minimize` documentation here: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html*

*Also, for your interest, similarly shaped potentials include the Ricker potential, which you can read more about here: https://en.wikipedia.org/wiki/Ricker_wavelet*

In [ ]:
#>>>EXERCISE: L4.5.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize as opt

def f(x):
    return #YOUR CODE HERE

sol=opt.minimize_scalar(f, bounds=(-6,6), method='bounded')
x = np.linspace(-6, 6, 100)

print('min value over the array x:',sol.x)

plt.xlabel('x', fontsize=15) #Label x
plt.ylabel('f(x)', fontsize=15)#Label y
plt.plot(x, f(x));
plt.axvline(sol.x, c='red')
plt.show()


>#### Follow-up 4.5.1a (ungraded)
>
>What makes this minimum particularly interesting? 

>#### Follow-up 4.5.1b (ungraded)
>
>Change the bounds of the minimizer. Does this lead to different results?

>#### Follow-up 4.5.1c (ungraded)
>
>Try changing the function to create an asymmetric potential (for example, add $x$). Does the minimizer find the global mimimum, or is it stuck in the local minimum? Do different methods (for instance `brent`) do a better or worse job?

<a name='section_4_6'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L4.6 Gradient Descent</h2>  

| [Top](#section_4_0) | [Previous Section](#section_4_5) | [Exercises](#exercises_4_6) | [Next Section](#section_4_7) |


In [ ]:
#>>>RUN: L4.6-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L04/slides_L04_06.html', width=975, height=550)

In [ ]:
#>>>RUN: L4.6-runcell01

#Let's find the minimum of x^2
def f(x):
    return x**2
# We need to define the gradient of f
def grad(x):
    return 2*x

#Simple gradient descent (this one is not physics-based, but rather just steps along the gradient)
#Starting on x, move in the direction of negative gradient, scaled by alpha (delta t from earlier)
def gd(x, grad, alpha, max_iter=10):
    xs = np.zeros(1 + max_iter)
    xs[0] = x #start at x
    for i in range(max_iter):
        x = x - alpha * grad(x)
        xs[i+1] = x
    return xs

#Physics based gradient descent, which comes from our equations above
#Now will update both position and velocity in timesteps alpha
def gd_momentum_nodamp(x, grad, alpha, max_iter=10):
    xs = np.zeros(1 + max_iter)
    xs[0] = x
    v = 0
    for i in range(max_iter):
        v = v - alpha*grad(x)
        x = x + alpha * v  - 0.5 * alpha * alpha*grad(x)
        xs[i+1] = x
    return xs

def plotGDAlgo(iGDAlgo,alpha=0.1,beta=None,x0=1):
    if beta is not None:
        xs = iGDAlgo(x0, grad, alpha, beta)
    else:
        xs = iGDAlgo(x0, grad, alpha)
    xp = np.linspace(-1.2, 1.2, 100)

    #Now just plotting code
    plt.xlabel('x', fontsize=15) #Label x
    plt.ylabel('f(x)', fontsize=15)#Label y
    plt.plot(xp, f(xp)) #function
    plt.plot(xs, f(xs), 'o-', c='red') #varied function
    for i, (x, y) in enumerate(zip(xs, f(xs)), 1):
        plt.text(x, y+0.2, i,bbox=dict(facecolor='yellow', alpha=0.5), fontsize=14)
    plt.show()    

plotGDAlgo(gd)
plotGDAlgo(gd,0.95) #large timestep alpha
plotGDAlgo(gd_momentum_nodamp,0.95) #Large timestep, with momentum

In [ ]:
#>>>RUN: L4.6-runcell02

#Now will update both momentum and velocity in timesteps alpha but with a dilution factor of beta
def gd_momentum(x, grad, alpha, beta=0.9, max_iter=10):
    xs = np.zeros(1 + max_iter)
    xs[0] = x
    v = 0
    for i in range(max_iter):
        v = beta*v + (1-beta)*grad(x) # beta effectively scales how much the velocity is affected by the local gradient
        vc = v/(1+beta**(i+1)) # and also controls the damping of the velocity
        x = x - alpha * vc
        xs[i+1] = x
    return xs

plotGDAlgo(gd)
plotGDAlgo(gd,0.95) #large timestep alpha
plotGDAlgo(gd_momentum_nodamp,0.95) #Large timestep
plotGDAlgo(gd_momentum,0.95) #Large timestep
plotGDAlgo(gd_momentum,0.95,beta=0.7) #Large timestep
plotGDAlgo(gd_momentum,0.95,beta=0.9) #Large timestep

In [ ]:
#>>>RUN: L4.6-runcell03

#Now let's do this with a general tool that optimizes
import scipy.linalg as la
def f(x):
    return x**4 + 3*(x-2)**3 - 15*(x)**2 + 1

def fprime(x):
    return 4*x**3 + 9*(x-2)**2 - 30*(x)

#Where is our minimizer
def custmin(fun, x0, args=(), maxfev=None, alpha=0.0002,
        maxiter=100000, tol=1e-10, callback=None, **options):
    """Implements simple gradient descent for the function above."""
    bestx = x0
    bestf = fun(x0)
    funcalls = 1
    niter = 0
    improved = True
    stop = False

    while improved and not stop and niter < maxiter:
        niter += 1
        # the next 2 lines are gradient descent
        step = alpha * fprime(bestx)
        bestx = bestx - step
        
        bestf = fun(bestx)
        funcalls += 1

        if la.norm(step) < tol:
            improved = False
        if callback is not None:
            callback(bestx)
        if maxfev is not None and funcalls >= maxfev:
            stop = True
            break

    return opt.OptimizeResult(fun=bestf, x=bestx, nit=niter,nfev=funcalls, success=(niter > 1))

def reporter(p):
    """Reporter function to capture intermediate states of optimization."""
    global ps
    ps.append(p)

x0=-7
x0 = np.array([x0])
ps = [x0]
sol=opt.minimize(f, x0, method=custmin, callback=reporter)
x = np.linspace(-8, 5, 100)
plt.xlabel('x', fontsize=15) #Label x
plt.ylabel('f(x)', fontsize=15)#Label y
plt.plot(x, f(x));
plt.axvline(sol.x, c='red')
plt.show()

#Now let's trick it
x0=5
x0 = np.array([x0])
ps = [x0]
sol=opt.minimize(f, x0, method=custmin, callback=reporter)
x = np.linspace(-8, 5, 100)
plt.xlabel('x', fontsize=15) #Label x
plt.ylabel('f(x)', fontsize=15)#Label y
plt.plot(x, f(x));
plt.axvline(sol.x, c='red')
plt.show()

<a name='exercises_4_6'></a>     

| [Top](#section_4_0) | [Restart Section](#section_4_6) | [Next Section](#section_4_7) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-4.6.1</span>

Let's say you were stuck using the simple minimizer that we developed above. Let's also assume that the global mimimum is within some range that you know. What could you do to make sure your minimizer finds the global minimum, instead of getting stuck in a local minimum? Select all that apply:

- Use a larger number of iterations.
- Use a smaller number for the tolerance.
- Choose random starting points for the minimizer, and save the result that yields the lowest value.
- Restart your minimizer once it finds it's first minimum, and use a new starting point that is some pre-defined (perhaps random) distance away from the minimum that is found; do this several times and save the result that yields the lowest value.


>#### Follow-up 4.6.1a (ungraded)
>
>Try some of the strategies above. What works? What does not?

<a name='section_4_7'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L4.7 Fitting the Full Range of Hubble Data</h2>  

| [Top](#section_4_0) | [Previous Section](#section_4_6) | [Exercises](#exercises_4_7) | [Next Section](#section_4_8) |


In [ ]:
#>>>RUN: L4.7-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L04/slides_L04_07.html', width=975, height=550)

In [ ]:
#>>>RUN: L4.7-runcell01

#Ok let's do a custom minimization of our fit function with gradient descent
#Note that since we are fitting two parameters we need to do this in 2D
redshift,distance,distance_err = load(label,10)
weights=np.array([])
for pVal in distance_err:
    weights = np.append(weights,1./pVal/pVal)

def f(x,h0,q):
    val=x*(1e6*3e5/h0)*(1 + ((1-q)*0.5)*x)
    return val

def fprime(x,h0,q):
    der=np.zeros(2)
    der[0]=-1*x*(1e6*3e5/h0/h0)*(1 + ((1-q)*0.5)*x)
    der[1]=x*(1e6*3e5/h0)*(-0.5*x)
    return der

def algof(inputs):
    d=0
    for i0 in range(len(redshift)):
        yhat=f(redshift[i0],inputs[0],inputs[1])
        pD=(distance[i0]-yhat)**2
        #pD=pD/dmean
        d+=pD*weights[i0]
    return d

def algofprime(inputs):
    d=np.zeros(2)
    for i0 in range(len(redshift)):
        yhat=f(redshift[i0],inputs[0],inputs[1])
        pD=2*(distance[i0]-yhat)
        yhatprime=fprime(redshift[i0],inputs[0],inputs[1])
        #print(yhatprime,pD,d)
        #pD=pD/dmean
        d=d+(yhatprime*pD)*weights[i0]
    return d

def custmin(fun, x0, args=(), maxfev=None, alpha=0.0001,maxiter=10000, tol=1e-10, callback=None, **options):
    """Implements simple gradient descent for the function above."""
    bestx = x0
    bestf = fun(x0)
    funcalls = 1
    niter = 0
    improved = True
    stop = False

    while improved and not stop and niter < maxiter:
        niter += 1
        if niter % 1000 == 0: 
            print(niter,bestx)
        # the next 2 lines are gradient descent
        step = alpha * algofprime(bestx)
        #print(bestx,step)
        bestx = bestx + step

        bestf = fun(bestx)
        funcalls += 1

        if la.norm(step) < tol:
            improved = False
        if callback is not None:
            callback(bestx)
        if maxfev is not None and funcalls >= maxfev:
            stop = True
            break

    return opt.OptimizeResult(fun=bestf, x=bestx, nit=niter,nfev=funcalls, success=(niter > 1))

def reporter(p):
    """Reporter function to capture intermediate states of optimization."""
    global ps
    ps.append(p)

#tmp=algofprime([70,0.03])
x0 = np.array([40,0])
ps = [x0]
sol0=opt.minimize(algof, x0, method=custmin, callback=reporter)
print(sol0)
print()

sol1=opt.minimize(algof, x0)#, method=custmin, callback=reporter)
print(sol1)
    

In [ ]:
#>>>RUN: L4.7-runcell02

#Ok let's do a custom minimization of our fit function with gradient descent
#Note that since we are fitting two parameters we need to do this in 2D
def f(x,h0,q):
    val=x*(1e6*3e5/h0)*(1 + ((1-q)*0.5)*x)
    return val

def algof(inputs):
    d=0
    for i0 in range(len(redshift)):
        yhat=f(redshift[i0],inputs[0],inputs[1])
        pD=(distance[i0]-yhat)**2
        #pD=pD/dmean
        d+=pD*weights[i0]
    return d

def algofprime(inputs):
    delta1=np.array([inputs[0]*0.001,0])
    delta2=np.array([0,inputs[1]*0.001])
    dp11=algof(inputs-delta1)
    dp21=algof(inputs-delta2)
    dp12=algof(inputs+delta1)
    dp22=algof(inputs+delta2)
    deriv=np.array([0,0])
    deriv[0]=(dp11-dp12)/(2*delta1[0])
    deriv[1]=(dp21-dp22)/(2*delta2[1])
    return deriv

def algofprime2(inputs):
    d=np.zeros(2)
    for i0 in range(len(redshift)):
        yhat=f(redshift[i0],inputs[0],inputs[1])
        pD=2*(distance[i0]-yhat)
        yhatprime=fprime(redshift[i0],inputs[0],inputs[1])
        #print(yhatprime,pD,d)
        #pD=pD/dmean
        d=d+(yhatprime*pD)*weights[i0]
    return d

def custmin(fun, x0, args=(), maxfev=None, alpha=0.01,maxiter=10000, tol=1e-10, callback=None, **options):
    """Implements simple gradient descent for the function above."""
    bestx = x0
    bestf = fun(x0)
    funcalls = 1
    niter = 0
    improved = True
    stop = False

    while improved and not stop and niter < maxiter:
        niter += 1
        if niter % 1000 == 0: 
            print(niter,bestx)
        # the next 2 lines are gradient descent
        step = alpha * algofprime(bestx)
        #print(bestx,step)
        bestx = bestx + step

        bestf = fun(bestx)
        funcalls += 1

        if la.norm(step) < tol:
            improved = False
        if callback is not None:
            callback(bestx)
        if maxfev is not None and funcalls >= maxfev:
            stop = True
            break

    return opt.OptimizeResult(fun=bestf, x=bestx, nit=niter,nfev=funcalls, success=(niter > 1))

def reporter(p):
    """Reporter function to capture intermediate states of optimization."""
    global ps
    ps.append(p)

x0 = np.array([40,0.1])
ps = [x0]
sol0=opt.minimize(algof, x0, method=custmin, callback=reporter)
print(sol0)
print()

sol1=opt.minimize(algof, x0)#, method=custmin, callback=reporter)
print(sol1)

<a name='exercises_4_7'></a>     

| [Top](#section_4_0) | [Restart Section](#section_4_7) | [Next Section](#section_4_8) |


>#### Follow-up 4.7.1a (ungraded)
>
>The `custmin` minimizer in the code cell `L4.7-runcell02` is not working properly. Can you get it to work? Is it faster than defining the derivative analytically?

<a name='section_4_8'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L4.8 Fitting with lmfit</h2>     

| [Top](#section_4_0) | [Previous Section](#section_4_7) | [Exercises](#exercises_4_8) |


In [ ]:
#>>>RUN: L4.8-runcell01

import lmfit

weights=np.array([])
for pVal in distance_err:
    #weighted fits in lmfit require you to ass in 1/sigma (not 1/sigma^2
    weights = np.append(weights,1./pVal)

#Clearly that's not working so lets use an approximation to this
def f(x,h0,q):
    val=x*(1e6*3e5/h0)*(1 + ((1-q)*0.5)*x)
    return val

model  = lmfit.Model(f)
p = model.make_params(h0=50,q=0)
result = model.fit(data=distance, params=p, x=redshift, weights=weights)
lmfit.report_fit(result)
plt.figure()
result.plot();

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-4.8.1 Modified Hubble Fit</span>

Add a constant term to the fit, and see if the fitted parameter is consistent with zero. Your fit function should have the following form:   

$$ f(x) = c + \frac{x}{h_{0}}\left(1+\frac{1-q}{2}x\right) $$

Run the fit with this modified form. What is your value for the constant $c$ in units of megaparsecs (Mpc)? How does this compare with the typical distance between galaxies?

Report your answer with precision 1e-2.

In [ ]:
#>>>EXERCISE: L4.8.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def f(x,h0,q,c):
    return #YOUR CODE HERE


###FILL IN THE CODE BELOW###

model = #YOUR CODE HERE
p = #YOUR CORE HERE 

######

model  = lmfit.Model(f)
p = model.make_params(h0=50,q=0,c=0)

result = model.fit(data=distance, params=p, x=redshift, weights=weights)
lmfit.report_fit(result)
plt.figure()
result.plot();
